In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('urlset/urlset.csv', error_bad_lines=False, encoding='latin1')

b'Skipping line 18259: expected 14 fields, saw 15\nSkipping line 18273: expected 14 fields, saw 15\n'
/Users/jackheavey/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.columns

Index(['domain', 'ranking', 'mld_res', 'mld.ps_res', 'card_rem', 'ratio_Rrem',
       'ratio_Arem', 'jaccard_RR', 'jaccard_RA', 'jaccard_AR', 'jaccard_AA',
       'jaccard_ARrd', 'jaccard_ARrem', 'label'],
      dtype='object')

In [4]:
df = df[['domain', 'label']]

In [15]:
#FEATURE IDEAS:
# 1) if the domain meets one of the top 10k(?) domains
# 2) number of directories traveled through (signified by /s)
# 3) Type of domain (.edu, .com, etc.) should be fairly predictive?

In [20]:
top1m = pd.read_csv('top-1m/top-1m.csv', nrows=1000, header=None)
top1m.columns = ['rank', 'url']
top10k = top1m['url'].tolist()
top = []
for i in top10k:
    x = i.split('.')[-2]
    if len(x) > 5:
        top.append(x)
top_final = []
top_final = [x for x in top if x not in top_final]

In [21]:
top_final

['google',
 'netflix',
 'netflix',
 'netflix',
 'netflix',
 'microsoft',
 'google',
 'microsoft',
 'netflix',
 'netflix',
 'netflix',
 'netflix',
 'netflix',
 'netflix',
 'microsoft',
 'windowsupdate',
 'windowsupdate',
 'googleapis',
 'microsoft',
 'googleapis',
 'microsoftonline',
 'microsoftonline',
 'doubleclick',
 'doubleclick',
 'facebook',
 'googleapis',
 'office',
 'googleapis',
 'googleusercontent',
 'google',
 'netflix',
 'netflix',
 'amazonaws',
 'google-analytics',
 'microsoft',
 'office365',
 'akadns',
 'microsoft',
 'google-analytics',
 'doubleclick',
 'akadns',
 'microsoft',
 'office365',
 'googleapis',
 'icloud',
 'akamaiedge',
 'digicert',
 'microsoft',
 'youtube',
 'office',
 'google',
 'microsoft',
 'aaplimg',
 'microsoft',
 'google',
 'digicert',
 'microsoft',
 'office',
 'aaplimg',
 'apple-dns',
 'googleapis',
 'microsoft',
 'microsoft',
 'microsoft',
 'googletagmanager',
 'googletagmanager',
 'office',
 'apple-dns',
 'windows',
 'google',
 'googleusercontent',
 'g

In [34]:
#######FUNCTION CELL

def get_domain(string):
    return string.rpartition('/')[0]

def in_top_10k(string):
    for i in top10k:
        if i in string:
            return 1
    return 0

def subdirectory_array(row):
    lst = []
    for i in top_final:
        if i in row['directory']:
            lst.append(1)
        else:
            lst.append(0)
    return pd.Series(lst)

In [35]:
#######Feature selection cell

df = pd.read_csv('urlset/urlset.csv', error_bad_lines=False, encoding='latin1')
df = df[['domain', 'label']]
df['top_level_site'] = df['domain'].apply(lambda row: get_domain(row))
df['directory'] = df['domain'].apply(lambda row: row.partition('/')[2])
df['in_top_10k'] = df['top_level_site'].apply(lambda row: in_top_10k(row))
df['number_of_directories'] = df['domain'].apply(lambda row: row.count('/'))
df['len'] = df['domain'].apply(lambda row: len(row))


df_apply = df.apply(lambda row: subdirectory_array(row), axis=1)
df = pd.concat([df, df_apply], axis=1)

b'Skipping line 18259: expected 14 fields, saw 15\nSkipping line 18273: expected 14 fields, saw 15\n'
/Users/jackheavey/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [36]:
df.head()

,domain,label,top_level_site,directory,in_top_10k,number_of_directories,len,0,1,2,...,766,767,768,769,770,771,772,773,774,775
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,1.0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,70ffb52d079109dca5664cce6f317373782/login.SkyP...,0,10,225,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,1.0,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,paypal.co.uk/cycgi-bin/webscrcmd=_home-custome...,0,4,81,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,1.0,serviciosbys.com/paypal.cgi.bin.get-into.herf....,paypal.cgi.bin.get-into.herf.secure.dispatch35...,0,11,177,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,mail.printakid.com/www.online.americanexpress....,1.0,mail.printakid.com/www.online.americanexpress.com,www.online.americanexpress.com/index.html,0,2,60,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,thewhiskeydregs.com/wp-content/themes/widescre...,1.0,thewhiskeydregs.com/wp-content/themes/widescre...,wp-content/themes/widescreen/includes/temp/pro...,0,10,116,0,0,0,...,0,0,0,0,0,0,0,0,0,0
